In [ ]:
import json
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers

url = 'search-havasmedia-h7o3s6bmcgvqv6ubgvkamqmiwa.us-west-2.es.amazonaws.com'
port = 80
#url = 'localhost'
#port = 9200
es = Elasticsearch(url,port=port)

def bulk_load(actions_):
    res = None
    try:
        res = helpers.parallel_bulk(es, actions_)
    except Exception as e:
        print("There was an error")
        print(e)
    return res

def process_result(result_list):
    success, fail = 0, 0
    for i in result_list:
        if i[0]:
            success = success + 1
        else:
            fail = fail + 1
    return success, fail

In [ ]:
indices = es.indices.get_alias("*")
query = {
    "query": {
        "match_all": {}
    }
}
for index in indices:
    indices[index] = es.count(index=index, body=query)['count']
indices

In [ ]:
del indices['mexico_tarifas']
del indices['colombia_agosto_2017_audio_reach']
del indices['mexico_diciembre_2017_video_view']
del indices['colombia_enero_2018_video_reach']
del indices['argentina_noviembre_2017_video_reach']
del indices['mexico_noviembre_2017_video_view']
del indices['colombia_agosto_2017_publishing_reach']
del indices['mexico_diciembre_2017_video_reach']
del indices['mexico_noviembre_2017_video_reach']
del indices['colombia_agosto_2017_video_reach']
del indices['chile_mayo_2017_video_reach']
indices

In [ ]:

docs = []
with open('/home/kira/projects/vot/backup.txt', 'w') as _file:
    for index,count in indices.items():
        query['size']=count
        hits = es.search(index=index, body=query)['hits']['hits']
        for i in hits:
            del i['_score']
        docs.append(hits)
    _file.write(json.dumps(docs,indent=4))
    

In [ ]:
data=open('/home/kira/projects/vot/backup.txt').read()
data = json.loads(data)
bulk_result = list(bulk_load(data))
result = process_result(bulk_result)

In [ ]:
def get_all_docs(index, query, total):
    max_window = 10000
    docs = []
    for i in range(max_window,total,max_window):
        query['from']=i-max_window
        query['size']=i
        print(query['from'], query['size'])
        hits = es.search(index=index, body=query)['hits']['hits']
        docs.append(hits)
    return docs


In [ ]:
def scroll_docs(index, query):
    max_window = 10000
    page = es.search(
        index = index,
        scroll = '10m',
        size = max_window,
        body = query)
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']
    docs = page['hits']['hits']
    while (scroll_size > 0):
        print(scroll_size)
        page = es.scroll(scroll_id = sid, scroll = '2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        hits = page['hits']['hits']
        scroll_size = len(hits)
        docs += hits
    return docs

In [ ]:
query = {
    "query": {
        "match_all": {}
    }
}
docs = scroll_docs('mexico_noviembre_2017_video_reach', query)

In [ ]:
def get_backup_indices(indices):
    for index,total in indices.items():
        docs = scroll_docs(index, query)
        with open('/home/kira/projects/vot/'+index, 'w') as _file:
            _file.write(json.dumps(docs,indent=4))

In [ ]:
query = {
    "query": {
        "match_all": {}
    }
}
get_backup_indices(indices)

In [ ]:
indices

In [ ]:
len(docs)

In [ ]:
with open('/home/kira/projects/vot/mexico_noviembre_2017_video_reach', 'w') as _file:
    _file.write(json.dumps(docs,indent=4))